BENV0093 TBPS6 - WLC Multi-Criteria Analysis
Weighted Linear Combination for wind farm site selection

WLC Data Processing

In [1]:
import numpy as np
import rasterio
from pathlib import Path

print("="*80)
print("WLC MULTI-CRITERIA ANALYSIS (5 CRITERIA)")
print("Updated Weight Configuration")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

WIND_TIF = OUTPUT_DIR / 'wind_suitability_score_original.tif'
GRID_TIF = OUTPUT_DIR / 'grid_suitability_score.tif'
ROADS_TIF = OUTPUT_DIR / 'roads_suitability_score.tif'
SLOPE_TIF = OUTPUT_DIR / 'slope_suitability_score.tif'
SATURATION_TIF = OUTPUT_DIR / 'saturation_suitability_score.tif'

WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'

# Updated weights (5 criteria)
WEIGHTS = {
    'Wind Speed': 0.3475,              
    'Proximity to Grid': 0.3475,       
    'Proximity to Roads': 0.1200,      
    'Slope': 0.0650,                   
    'Grid Integration Risk': 0.1200    
}

print(f"\nCriteria Weights (5 criteria):")
for criterion, weight in WEIGHTS.items():
    print(f"  {criterion:25s}: {weight:.4f} ({weight*100:.2f}%)")

total_weight = sum(WEIGHTS.values())
print(f"\nTotal Weight: {total_weight:.4f} ({total_weight*100:.2f}%)")

if abs(total_weight - 1.0) > 0.001:
    print("  WARNING: Weights do not sum to 1.0!")
else:
    print("  ✓ Weights sum to 100%")

# Step 1: Load all criteria
print("\n" + "="*80)
print("STEP 1: LOAD CRITERIA LAYERS")
print("="*80)

criteria_data = {}
transform = None
rows, cols = None, None
extent = None

for name, filepath in [
    ('Wind Speed', WIND_TIF),
    ('Proximity to Grid', GRID_TIF),
    ('Proximity to Roads', ROADS_TIF),
    ('Slope', SLOPE_TIF),
    ('Grid Integration Risk', SATURATION_TIF)
]:
    print(f"\nLoading {name}...")
    
    if not filepath.exists():
        print(f"  ✗ ERROR: File not found: {filepath}")
        exit(1)
    
    with rasterio.open(filepath) as src:
        data = src.read(1).astype('float32')
        data[data == -9999] = np.nan
        
        if transform is None:
            transform = src.transform
            rows, cols = src.shape
            
            pixel_size_x = abs(transform.a)
            pixel_size_y = abs(transform.e)
            extent = [
                transform.c,
                transform.c + cols * pixel_size_x,
                transform.f - rows * pixel_size_y,
                transform.f
            ]
        
        criteria_data[name] = data
        
        valid = data[~np.isnan(data)]
        print(f"  ✓ Loaded: {rows}x{cols}")
        print(f"  Mean: {valid.mean():.2f}, Min: {valid.min():.2f}, Max: {valid.max():.2f}")
        print(f"  Valid cells: {len(valid):,}")

print(f"\nVerification:")
print(f"  Shape: {rows}x{cols}")
print(f"  Extent: X[{extent[0]:.0f}, {extent[1]:.0f}], Y[{extent[2]:.0f}, {extent[3]:.0f}]")

# Step 2: Calculate WLC
print("\n" + "="*80)
print("STEP 2: CALCULATE WLC SCORE (5 CRITERIA)")
print("="*80)

print(f"\nFormula:")
print(f"  WLC = {WEIGHTS['Wind Speed']:.4f}*Wind + " +
      f"{WEIGHTS['Proximity to Grid']:.4f}*Grid + " +
      f"{WEIGHTS['Proximity to Roads']:.4f}*Roads + " +
      f"{WEIGHTS['Slope']:.4f}*Slope + " +
      f"{WEIGHTS['Grid Integration Risk']:.4f}*Saturation")

wlc_score = (
    WEIGHTS['Wind Speed'] * criteria_data['Wind Speed'] +
    WEIGHTS['Proximity to Grid'] * criteria_data['Proximity to Grid'] +
    WEIGHTS['Proximity to Roads'] * criteria_data['Proximity to Roads'] +
    WEIGHTS['Slope'] * criteria_data['Slope'] +
    WEIGHTS['Grid Integration Risk'] * criteria_data['Grid Integration Risk']
)

land_mask = ~(
    np.isnan(criteria_data['Wind Speed']) |
    np.isnan(criteria_data['Proximity to Grid']) |
    np.isnan(criteria_data['Proximity to Roads']) |
    np.isnan(criteria_data['Slope']) |
    np.isnan(criteria_data['Grid Integration Risk'])
)

wlc_score[~land_mask] = np.nan

valid_wlc = wlc_score[~np.isnan(wlc_score)]

print(f"\nWLC Statistics:")
print(f"  Mean: {valid_wlc.mean():.2f}")
print(f"  Median: {np.median(valid_wlc):.2f}")
print(f"  Std: {valid_wlc.std():.2f}")
print(f"  Min: {valid_wlc.min():.2f}")
print(f"  Max: {valid_wlc.max():.2f}")
print(f"  Valid cells: {len(valid_wlc):,}")

suitability_classes = [
    ("Excellent", 8, 10.1),
    ("Good", 6, 8),
    ("Fair", 4, 6),
    ("Marginal", 2, 4),
    ("Poor", 0, 2)
]

print(f"\nSuitability Distribution:")
for label, s_min, s_max in suitability_classes:
    mask = (valid_wlc >= s_min) & (valid_wlc < s_max)
    count = mask.sum()
    pct = count / len(valid_wlc) * 100
    print(f"  {label:12s} ({s_min:.0f}-{s_max:.0f}): {count:5,} cells ({pct:6.2f}%)")

# Step 3: Save WLC raster
print("\n" + "="*80)
print("STEP 3: SAVE WLC RASTER")
print("="*80)

tif_meta = {
    'driver': 'GTiff',
    'height': rows,
    'width': cols,
    'count': 1,
    'dtype': 'float32',
    'crs': 'EPSG:27700',
    'transform': transform,
    'compress': 'lzw',
    'nodata': -9999
}

wlc_output = wlc_score.copy()
wlc_output[np.isnan(wlc_output)] = -9999

with rasterio.open(WLC_TIF, 'w', **tif_meta) as dst:
    dst.write(wlc_output.astype('float32'), 1)

print(f"✓ Saved: {WLC_TIF}")

print("\n" + "="*80)
print("WLC CALCULATION COMPLETE (5 CRITERIA)")
print("="*80)

print(f"""
Output File:
  - wlc_suitability_score_5criteria.tif

WLC Formula (Updated Weights):
  Score = {WEIGHTS['Wind Speed']:.4f}*Wind + 
          {WEIGHTS['Proximity to Grid']:.4f}*Grid + 
          {WEIGHTS['Proximity to Roads']:.4f}*Roads + 
          {WEIGHTS['Slope']:.4f}*Slope + 
          {WEIGHTS['Grid Integration Risk']:.4f}*Saturation

Criteria Weights:
  Wind Speed:             34.75%
  Proximity to Grid:      34.75%
  Proximity to Roads:     12.00%
  Slope:                   6.50%
  Grid Integration Risk:  12.00% (Regional Saturation)
  ─────────────────────────────
  Total:                 100.00%

Results:
  Mean Score: {valid_wlc.mean():.2f}
  Median Score: {np.median(valid_wlc):.2f}
  Std: {valid_wlc.std():.2f}
  Range: {valid_wlc.min():.2f} - {valid_wlc.max():.2f}

Distribution:
  Excellent (8-10): {(valid_wlc >= 8).sum() / len(valid_wlc) * 100:.1f}%
  Good (6-8): {((valid_wlc >= 6) & (valid_wlc < 8)).sum() / len(valid_wlc) * 100:.1f}%
  Fair (4-6): {((valid_wlc >= 4) & (valid_wlc < 6)).sum() / len(valid_wlc) * 100:.1f}%
  
All 5 criteria successfully combined!

""")

WLC MULTI-CRITERIA ANALYSIS (5 CRITERIA)
Updated Weight Configuration

Criteria Weights (5 criteria):
  Wind Speed               : 0.3475 (34.75%)
  Proximity to Grid        : 0.3475 (34.75%)
  Proximity to Roads       : 0.1200 (12.00%)
  Slope                    : 0.0650 (6.50%)
  Grid Integration Risk    : 0.1200 (12.00%)

Total Weight: 1.0000 (100.00%)
  ✓ Weights sum to 100%

STEP 1: LOAD CRITERIA LAYERS

Loading Wind Speed...
  ✓ Loaded: 235x131
  Mean: 1.13, Min: 0.00, Max: 10.00
  Valid cells: 10,397

Loading Proximity to Grid...
  ✓ Loaded: 235x131
  Mean: 5.71, Min: 0.00, Max: 10.00
  Valid cells: 9,758

Loading Proximity to Roads...
  ✓ Loaded: 235x131
  Mean: 5.86, Min: 0.00, Max: 10.00
  Valid cells: 9,758

Loading Slope...
  ✓ Loaded: 235x131
  Mean: 8.98, Min: 0.00, Max: 10.00
  Valid cells: 10,397

Loading Grid Integration Risk...
  ✓ Loaded: 235x131
  Mean: 7.24, Min: 0.00, Max: 10.00
  Valid cells: 9,758

Verification:
  Shape: 235x131
  Extent: X[0, 655000], Y[10000, 

WLC Map

In [2]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import geopandas as gpd
from pathlib import Path

print("="*80)
print("WLC VISUALIZATION (5 CRITERIA)")
print("Updated Weight Configuration")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'
WLC_MAP = OUTPUT_DIR / 'map10_wlc_suitability_5criteria.png'

# Updated weights for display
WEIGHTS = {
    'Wind Speed': 0.3475,              
    'Grid Distance': 0.3475,           
    'Road Proximity': 0.1200,          
    'Slope': 0.0650,                   
    'Grid Integration Risk': 0.1200    
}

# Step 1: Load WLC raster
print("\nStep 1: Loading WLC raster...")

if not WLC_TIF.exists():
    print(f"ERROR: WLC file not found: {WLC_TIF}")
    print("Please run wlc_5criteria_calculate_updated.py first!")
    exit(1)

with rasterio.open(WLC_TIF) as src:
    wlc_score = src.read(1).astype('float32')
    wlc_score[wlc_score == -9999] = np.nan
    
    transform = src.transform
    rows, cols = src.shape
    
    pixel_size_x = abs(transform.a)
    pixel_size_y = abs(transform.e)
    extent = [
        transform.c,
        transform.c + cols * pixel_size_x,
        transform.f - rows * pixel_size_y,
        transform.f
    ]

valid_wlc = wlc_score[~np.isnan(wlc_score)]

print(f"✓ Loaded: {rows}x{cols}")
print(f"  Mean: {valid_wlc.mean():.2f}, Min: {valid_wlc.min():.2f}, Max: {valid_wlc.max():.2f}")
print(f"  Valid cells: {len(valid_wlc):,}")

# Step 2: Calculate distribution
print("\nStep 2: Calculating suitability distribution...")

suitability_classes = [
    ("Excellent", 8, 10.1),
    ("Good", 6, 8),
    ("Fair", 4, 6),
    ("Marginal", 2, 4),
    ("Poor", 0, 2)
]

class_pcts = []
for label, s_min, s_max in suitability_classes:
    mask = (valid_wlc >= s_min) & (valid_wlc < s_max)
    pct = mask.sum() / len(valid_wlc) * 100
    class_pcts.append((label, pct))
    print(f"  {label:12s} ({s_min:.0f}-{s_max:.0f}): {pct:6.2f}%")

# Step 3: Load UK boundary
print("\nStep 3: Loading UK boundary...")

uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != 27700:
    uk_boundary = uk_boundary.to_crs('EPSG:27700')
print("✓ UK boundary loaded")

# Step 4: Generate WLC map
print("\nStep 4: Generating WLC map...")

fig, ax = plt.subplots(figsize=(14, 18), dpi=300)
ax.set_facecolor('#e6f2ff')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5, color='gray', zorder=1)

wlc_colors = [
    '#d73027', '#f46d43', '#fdae61', '#fee090', '#ffffbf',
    '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850', '#006837'
]
wlc_cmap = LinearSegmentedColormap.from_list('wlc', wlc_colors, N=256)
wlc_levels = np.arange(0, 10.5, 0.5)
wlc_norm = BoundaryNorm(wlc_levels, wlc_cmap.N, clip=True)

im = ax.imshow(wlc_score, cmap=wlc_cmap, norm=wlc_norm,
               extent=extent, origin='upper',
               interpolation='bilinear', aspect='equal', zorder=2)

uk_boundary.boundary.plot(ax=ax, color='black', linewidth=1.5, zorder=10)

ax.set_xlabel('Easting (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Northing (m)', fontsize=14, fontweight='bold')
ax.set_title('WLC Multi-Criteria Suitability for Wind Farm Development\n' +
             '(5 Criteria: Wind Speed, Grid, Roads, Slope, Grid Integration Risk)',
             fontsize=18, fontweight='bold', pad=20)

ax.ticklabel_format(style='plain')
ax.tick_params(labelsize=11)

cbar = plt.colorbar(im, ax=ax, pad=0.02, fraction=0.046, ticks=[0, 2, 4, 6, 8, 10])
cbar.set_label('Overall Suitability Score', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

threshold_info = [
    (0, 'Unsuitable', '#d73027'),
    (2, 'Poor', '#fdae61'),
    (4, 'Marginal', '#ffffbf'),
    (6, 'Fair', '#a6d96a'),
    (8, 'Good', '#1a9850'),
    (10, 'Excellent', '#006837'),
]

for score, label, color in threshold_info:
    cbar.ax.axhline(y=score, color=color, linestyle='--', linewidth=1.5, alpha=0.7)
    cbar.ax.text(1.3, score, label, fontsize=8, va='center',
                 color=color, fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8,
                          edgecolor=color, linewidth=1))

legend_elements = [
    mpatches.Patch(color='#006837', label=f'Excellent (8-10): {class_pcts[0][1]:.1f}%'),
    mpatches.Patch(color='#66bd63', label=f'Good (6-8): {class_pcts[1][1]:.1f}%'),
    mpatches.Patch(color='#a6d96a', label=f'Fair (4-6): {class_pcts[2][1]:.1f}%'),
    mpatches.Patch(color='#ffffbf', label=f'Marginal (2-4): {class_pcts[3][1]:.1f}%'),
    mpatches.Patch(color='#d73027', label=f'Poor (0-2): {class_pcts[4][1]:.1f}%'),
]

legend = ax.legend(handles=legend_elements, loc='upper left',
                   fontsize=10, framealpha=0.95, edgecolor='black',
                   title='Suitability Classes', title_fontsize=11,
                   fancybox=True, shadow=True)
legend.get_frame().set_linewidth(1.5)

dist_text = "Distribution:\n"
for label, pct in class_pcts:
    dist_text += f"{label:>9s}: {pct:5.1f}%\n"

stats_text = f"""WLC Statistics:
Mean: {valid_wlc.mean():.2f}
Median: {np.median(valid_wlc):.2f}
Std: {valid_wlc.std():.2f}
Min: {valid_wlc.min():.2f}
Max: {valid_wlc.max():.2f}

{dist_text}
Criteria Weights:
Wind Speed:      34.75%
Grid Dist:       34.75%
Road Prox:       12.00%
Slope:            6.50%
Grid Int. Risk:  12.00%

5-Criteria Model
Total: 100.0%

Grid: {rows}x{cols}
Valid: {len(valid_wlc):,} cells"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95,
             edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.15, stats_text, transform=ax.transAxes,
        fontsize=8.5, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.05
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000

ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x, scale_x], [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.plot([scale_x + scale_length, scale_x + scale_length],
        [scale_y - 5000, scale_y + 5000], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 12000, '100 km',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.9), zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.08
ax.annotate('N', xy=(arrow_x, arrow_y + 40000), xytext=(arrow_x, arrow_y),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=16, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(WLC_MAP, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ Saved: {WLC_MAP}")
plt.close()

print("\n" + "="*80)
print("WLC VISUALIZATION COMPLETE")
print("="*80)

print(f"""
Output File:
  - map10_wlc_suitability_5criteria.png

Map Details:
  - Colormap: Red (poor) to Green (excellent)
  - Scale: 0-10 suitability score
  - Resolution: {rows}x{cols} grid
  - Projection: British National Grid (EPSG:27700)

Criteria Weights (Updated):
  Wind Speed:             34.75%
  Proximity to Grid:      34.75%
  Proximity to Roads:     12.00%
  Slope:                   6.50%
  Grid Integration Risk:  12.00% (Saturation)

Statistics displayed on map:
  Mean: {valid_wlc.mean():.2f}
  Distribution across 5 suitability classes
  All 5 criteria weights shown

Map saved successfully!
""")

WLC VISUALIZATION (5 CRITERIA)
Updated Weight Configuration

Step 1: Loading WLC raster...
✓ Loaded: 235x131
  Mean: 4.47, Min: 0.47, Max: 8.77
  Valid cells: 9,743

Step 2: Calculating suitability distribution...
  Excellent    (8-10):   0.04%
  Good         (6-8):  11.52%
  Fair         (4-6):  53.97%
  Marginal     (2-4):  30.83%
  Poor         (0-2):   3.64%

Step 3: Loading UK boundary...
✓ UK boundary loaded

Step 4: Generating WLC map...
✓ Saved: D:\BENV0093\outputs\map10_wlc_suitability_5criteria.png

WLC VISUALIZATION COMPLETE

Output File:
  - map10_wlc_suitability_5criteria.png

Map Details:
  - Colormap: Red (poor) to Green (excellent)
  - Scale: 0-10 suitability score
  - Resolution: 235x131 grid
  - Projection: British National Grid (EPSG:27700)

Criteria Weights (Updated):
  Wind Speed:             34.75%
  Proximity to Grid:      34.75%
  Proximity to Roads:     12.00%
  Slope:                   6.50%
  Grid Integration Risk:  12.00% (Saturation)

Statistics displayed o

Apply Hard Constraints to WLC
Applies combined constraint mask to WLC suitability

In [3]:
import numpy as np
import rasterio
from pathlib import Path

print("="*80)
print("APPLY HARD CONSTRAINTS TO WLC (5 CRITERIA)")
print("Updated Weight Configuration")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

WLC_TIF = OUTPUT_DIR / 'wlc_suitability_score_5criteria.tif'
COMBINED_MASK_TIF = OUTPUT_DIR / 'hard_constraints_combined_mask.tif'
WIND_SPEED_TIF = OUTPUT_DIR / 'wind_speed_100m_original.tif'  

WLC_CONSTRAINED_TIF = OUTPUT_DIR / 'wlc_suitability_constrained_5criteria.tif'

NODATA_VALUE = -9999

# Step 1: Load WLC
print("\nStep 1: Loading WLC suitability score (5 criteria)")

if not WLC_TIF.exists():
    print(f"ERROR: WLC file not found: {WLC_TIF}")
    print("Please run wlc_5criteria_calculate_updated.py first!")
    exit(1)

with rasterio.open(WLC_TIF) as src:
    wlc_data = src.read(1)
    ref_meta = src.meta.copy()
    
    rows, cols = src.shape
    land_mask = (wlc_data != NODATA_VALUE) & (~np.isnan(wlc_data))

original_valid = land_mask.sum()
valid_original = wlc_data[land_mask]

print(f"Grid: {rows} x {cols}")
print(f"Original valid cells: {original_valid:,}")
print(f"Original WLC mean: {valid_original.mean():.2f}")
print(f"Criteria weights: Wind 34.75%, Grid 34.75%, Roads 12%, Slope 6.5%, Integration Risk 12%")

# NEW: Step 1b: Load wind speed data
print("\nStep 1b: Loading wind speed data")

if not WIND_SPEED_TIF.exists():
    print(f"WARNING: Wind speed file not found: {WIND_SPEED_TIF}")
    print("Skipping wind speed analysis...")
    wind_speed_available = False
else:
    with rasterio.open(WIND_SPEED_TIF) as src:
        wind_speed_data = src.read(1)
        wind_speed_data[wind_speed_data == NODATA_VALUE] = np.nan
        wind_speed_available = True
    
    valid_wind_original = wind_speed_data[land_mask & ~np.isnan(wind_speed_data)]
    print(f"✓ Wind speed data loaded")
    print(f"  Original area mean wind speed: {valid_wind_original.mean():.2f} m/s")
    print(f"  Original area median wind speed: {np.median(valid_wind_original):.2f} m/s")
    print(f"  Original area std: {valid_wind_original.std():.2f} m/s")

# Step 2: Load combined mask
print("\nStep 2: Loading combined constraint mask")

if not COMBINED_MASK_TIF.exists():
    print(f"ERROR: Constraint mask not found: {COMBINED_MASK_TIF}")
    print("Please run the constraint mask generation script first!")
    exit(1)

with rasterio.open(COMBINED_MASK_TIF) as src:
    combined_mask = src.read(1)
    excluded_mask = (combined_mask == 1)

excluded_cells = excluded_mask.sum()

print(f"✓ Constraint mask loaded")
print(f"  Cells to exclude: {excluded_cells:,} ({excluded_cells/land_mask.sum()*100:.1f}%)")

# Step 3: Apply constraints to WLC
print("\nStep 3: Applying constraints to WLC")

wlc_constrained = wlc_data.copy()
wlc_constrained[excluded_mask] = np.nan

constrained_valid = (~np.isnan(wlc_constrained) & land_mask).sum()
valid_constrained = wlc_constrained[~np.isnan(wlc_constrained) & land_mask]

print(f"After constraints: {constrained_valid:,} cells")
print(f"Excluded: {original_valid - constrained_valid:,} cells ({(original_valid - constrained_valid)/original_valid*100:.1f}%)")

# NEW: Step 3b: Calculate wind speed statistics for remaining area
if wind_speed_available:
    print("\nStep 3b: Wind speed statistics for remaining area")
    
    # Create constrained wind speed mask
    constrained_land_mask = land_mask & ~excluded_mask
    valid_wind_constrained = wind_speed_data[constrained_land_mask & ~np.isnan(wind_speed_data)]
    
    print(f"  Remaining area mean wind speed: {valid_wind_constrained.mean():.2f} m/s")
    print(f"  Remaining area median wind speed: {np.median(valid_wind_constrained):.2f} m/s")
    print(f"  Remaining area std: {valid_wind_constrained.std():.2f} m/s")
    print(f"  Remaining area min: {valid_wind_constrained.min():.2f} m/s")
    print(f"  Remaining area max: {valid_wind_constrained.max():.2f} m/s")
    
    # Wind speed change analysis
    wind_speed_change = valid_wind_constrained.mean() - valid_wind_original.mean()
    print(f"\n  Wind speed change after constraints: {wind_speed_change:+.2f} m/s")
    
    # Wind speed distribution in remaining area
    print(f"\n  Wind speed distribution in remaining area:")
    for threshold in [6, 7, 8, 9, 10]:
        count = (valid_wind_constrained >= threshold).sum()
        pct = count / len(valid_wind_constrained) * 100
        print(f"    ≥{threshold} m/s: {count:7,} cells ({pct:5.1f}%)")

# Step 4: Statistics comparison
print("\nStep 4: WLC statistics comparison")

print(f"\nOriginal WLC (5 criteria, 12% Integration Risk):")
print(f"  Valid cells: {original_valid:,}")
print(f"  Mean: {valid_original.mean():.2f}")
print(f"  Median: {np.median(valid_original):.2f}")
print(f"  Std: {valid_original.std():.2f}")
print(f"  Min: {valid_original.min():.2f}")
print(f"  Max: {valid_original.max():.2f}")

print(f"\nConstrained WLC (5 criteria, 12% Integration Risk):")
print(f"  Valid cells: {constrained_valid:,}")
print(f"  Mean: {valid_constrained.mean():.2f}")
print(f"  Median: {np.median(valid_constrained):.2f}")
print(f"  Std: {valid_constrained.std():.2f}")
print(f"  Min: {valid_constrained.min():.2f}")
print(f"  Max: {valid_constrained.max():.2f}")

# Step 5: Suitability distribution
print("\nStep 5: Suitability distribution")

classes = [
    ("Excellent", 8, 10.1),
    ("Good", 6, 8),
    ("Fair", 4, 6),
    ("Marginal", 2, 4),
    ("Poor", 0, 2)
]

print("\nOriginal WLC distribution:")
for label, s_min, s_max in classes:
    mask = (valid_original >= s_min) & (valid_original < s_max)
    count = mask.sum()
    pct = count / len(valid_original) * 100
    print(f"  {label:12s} ({s_min:.0f}-{s_max:.0f}): {count:5d} cells ({pct:5.1f}%)")

print("\nConstrained WLC distribution:")
for label, s_min, s_max in classes:
    mask = (valid_constrained >= s_min) & (valid_constrained < s_max)
    count = mask.sum()
    pct = count / len(valid_constrained) * 100
    print(f"  {label:12s} ({s_min:.0f}-{s_max:.0f}): {count:5d} cells ({pct:5.1f}%)")

# Step 6: Save constrained WLC
print("\nStep 6: Saving constrained WLC")

wlc_output = wlc_constrained.copy()
wlc_output[np.isnan(wlc_output)] = NODATA_VALUE

ref_meta.update({
    'dtype': 'float32',
    'nodata': NODATA_VALUE
})

with rasterio.open(WLC_CONSTRAINED_TIF, 'w', **ref_meta) as dst:
    dst.write(wlc_output.astype('float32'), 1)

print(f"✓ Saved: {WLC_CONSTRAINED_TIF}")

# Step 7: Verify alignment
print("\nStep 7: Verifying alignment")
with rasterio.open(WLC_TIF) as ref, rasterio.open(WLC_CONSTRAINED_TIF) as constrained:
    checks = {
        'Shape': ref.shape == constrained.shape,
        'Transform': ref.transform.almost_equals(constrained.transform, precision=0.01),
        'CRS': ref.crs == constrained.crs,
    }
    
    for check, result in checks.items():
        status = "✓ MATCH" if result else "✗ MISMATCH"
        print(f"  {check}: {status}")
    
    if all(checks.values()):
        print("\n✓ PERFECT ALIGNMENT")
    else:
        print("\n  ALIGNMENT ISSUES DETECTED")

print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)

# NEW: Enhanced summary with wind speed information
if wind_speed_available:
    wind_summary = f"""
  Wind Resource Analysis:
    Original area mean wind speed: {valid_wind_original.mean():.2f} m/s
    Remaining area mean wind speed: {valid_wind_constrained.mean():.2f} m/s
    Change after constraints: {wind_speed_change:+.2f} m/s
    
    Remaining area wind speed distribution:
      ≥6 m/s: {((valid_wind_constrained >= 6).sum()/len(valid_wind_constrained)*100):.1f}%
      ≥7 m/s: {((valid_wind_constrained >= 7).sum()/len(valid_wind_constrained)*100):.1f}%
      ≥8 m/s: {((valid_wind_constrained >= 8).sum()/len(valid_wind_constrained)*100):.1f}%
"""
else:
    wind_summary = """
  Wind Resource Analysis:
    Wind speed data not available
"""

print(f"""
Summary:
  Original WLC (5 criteria, updated weights):
    Valid cells: {original_valid:,}
    Mean score: {valid_original.mean():.2f}
    
    Criteria Weights:
      Wind Speed:             34.75%
      Proximity to Grid:      34.75%
      Proximity to Roads:     12.00%
      Slope:                   6.50%
      Grid Integration Risk:  12.00% (Saturation)
  
  After hard constraints:
    Valid cells: {constrained_valid:,}
    Mean score: {valid_constrained.mean():.2f}
  
  Exclusion:
    Cells excluded: {original_valid - constrained_valid:,}
    Exclusion rate: {(original_valid - constrained_valid)/original_valid*100:.1f}%
    Remaining: {constrained_valid/original_valid*100:.1f}%
  
  Constrained distribution:
    Excellent (>=8): {((valid_constrained >= 8).sum()/len(valid_constrained)*100):.1f}%
    Good (6-8): {(((valid_constrained >= 6) & (valid_constrained < 8)).sum()/len(valid_constrained)*100):.1f}%
    Fair (4-6): {(((valid_constrained >= 4) & (valid_constrained < 6)).sum()/len(valid_constrained)*100):.1f}%
{wind_summary}
Hard constraints applied:
  • Protected areas excluded
  • 500m settlement buffer excluded
  • Water bodies excluded

Output: {WLC_CONSTRAINED_TIF.name}

""")

APPLY HARD CONSTRAINTS TO WLC (5 CRITERIA)
Updated Weight Configuration

Step 1: Loading WLC suitability score (5 criteria)
Grid: 235 x 131
Original valid cells: 9,743
Original WLC mean: 4.47
Criteria weights: Wind 34.75%, Grid 34.75%, Roads 12%, Slope 6.5%, Integration Risk 12%

Step 1b: Loading wind speed data
✓ Wind speed data loaded
  Original area mean wind speed: 5.84 m/s
  Original area median wind speed: 5.62 m/s
  Original area std: 0.89 m/s

Step 2: Loading combined constraint mask
✓ Constraint mask loaded
  Cells to exclude: 7,144 (73.3%)

Step 3: Applying constraints to WLC
After constraints: 2,599 cells
Excluded: 7,144 cells (73.3%)

Step 3b: Wind speed statistics for remaining area
  Remaining area mean wind speed: 6.30 m/s
  Remaining area median wind speed: 6.10 m/s
  Remaining area std: 1.02 m/s
  Remaining area min: 3.80 m/s
  Remaining area max: 10.28 m/s

  Wind speed change after constraints: +0.46 m/s

  Wind speed distribution in remaining area:
    ≥6 m/s:   1,4

Identify Top 20 Optimal Wind Farm Sites (Mainland Only)
Selects best onshore sites with constraints:
- 35km minimum separation
- Roads distance < 30km (exclude offshore islands)

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from shapely.geometry import Point
from pathlib import Path
import pickle

print("="*80)
print("TOP 20 OPTIMAL ONSHORE WIND FARM SITES")
print("Based on 5-Criteria WLC (including Regional Saturation)")
print("With Wind Speed ≥ 6 m/s Threshold")
print("="*80)

BASE_DIR = Path('D:/BENV0093')
OUTPUT_DIR = BASE_DIR / 'outputs'

WLC_CONSTRAINED_TIF = OUTPUT_DIR / 'wlc_suitability_constrained_5criteria.tif'
REFERENCE_GRID_PKL = OUTPUT_DIR / 'reference_grid.pkl'
UK_BOUNDARY_SHP = BASE_DIR / 'UK_ITL1_Boundaries.shp'

# Original data layers (for raw values, not scores)
WIND_SPEED_TIF = OUTPUT_DIR / 'wind_speed_100m_original.tif'  
SLOPE_TIF_RAW = OUTPUT_DIR / 'slope_degrees.tif'  

# Score layers
WIND_SCORE_TIF = OUTPUT_DIR / 'wind_suitability_score_original.tif'
GRID_DIST_TIF = OUTPUT_DIR / 'grid_distance.tif'
ROADS_DIST_TIF = OUTPUT_DIR / 'roads_distance.tif'
SLOPE_SCORE_TIF = OUTPUT_DIR / 'slope_suitability_score.tif'
SATURATION_SCORE_TIF = OUTPUT_DIR / 'saturation_suitability_score.tif'

OUTPUT_CSV = OUTPUT_DIR / 'optimal_sites_top20_GB_5criteria.csv'
OUTPUT_DETAILED_CSV = OUTPUT_DIR / 'optimal_sites_top20_GB_5criteria_detailed.csv'
OUTPUT_MAP = OUTPUT_DIR / 'map_optimal_sites_top20_GB_5criteria.png'

MIN_SEPARATION_KM = 35
MAX_ROADS_DISTANCE_KM = 30
MIN_WIND_SPEED_MS = 6.0  
TOP_N = 20

# Northern Ireland exclusion boundary
NI_BOUNDARY = {
    'x_min': 0,
    'x_max': 200000,
    'y_min': 300000,
    'y_max': 600000
}

print(f"\nSelection Parameters:")
print(f"  Top N sites: {TOP_N}")
print(f"  Minimum separation: {MIN_SEPARATION_KM} km")
print(f"  Maximum roads distance: {MAX_ROADS_DISTANCE_KM} km")
print(f"  Minimum wind speed: {MIN_WIND_SPEED_MS} m/s (economic viability)")
print(f"  Northern Ireland: EXCLUDED")
print(f"  Geographic scope: Great Britain (England, Scotland, Wales)")

# Step 1: Load reference grid
print("\n" + "="*80)
print("STEP 1: LOAD REFERENCE GRID")
print("="*80)

with open(REFERENCE_GRID_PKL, 'rb') as f:
    ref = pickle.load(f)

rows, cols = ref['shape']
pixel_size = ref['pixel_size']
extent = ref['extent']

x_min, y_min = extent[0], extent[2]
y_max = extent[3]
x_spacing, y_spacing = pixel_size

print(f"Grid: {rows} x {cols}")
print(f"Pixel size: {x_spacing}m x {y_spacing}m")

# Step 2: Load constrained WLC
print("\n" + "="*80)
print("STEP 2: LOAD CONSTRAINED WLC (5 CRITERIA)")
print("="*80)

with rasterio.open(WLC_CONSTRAINED_TIF) as src:
    wlc_data = src.read(1)
    wlc_data[wlc_data == -9999] = np.nan

valid_cells = (~np.isnan(wlc_data)).sum()
print(f"Valid cells: {valid_cells:,}")
print(f"WLC range: {np.nanmin(wlc_data):.2f} - {np.nanmax(wlc_data):.2f}")

# Step 3: Load all data layers
print("\n" + "="*80)
print("STEP 3: LOAD DATA LAYERS")
print("="*80)

criteria_data = {}

# Load ORIGINAL wind speed (m/s)
print("Loading Wind Speed (m/s)...")
with rasterio.open(WIND_SPEED_TIF) as src:
    wind_speed_ms = src.read(1)
    wind_speed_ms[wind_speed_ms == -9999] = np.nan
    criteria_data['wind_speed_ms'] = wind_speed_ms

# Load wind speed SCORE
print("Loading Wind Score...")
with rasterio.open(WIND_SCORE_TIF) as src:
    wind_score = src.read(1)
    wind_score[wind_score == -9999] = np.nan
    criteria_data['wind_score'] = wind_score

# Load ORIGINAL slope (degrees)
print("Loading Slope (degrees)...")
with rasterio.open(SLOPE_TIF_RAW) as src:
    slope_degrees = src.read(1)
    slope_degrees[slope_degrees == -9999] = np.nan
    criteria_data['slope_degrees'] = slope_degrees

# Load slope SCORE
print("Loading Slope Score...")
with rasterio.open(SLOPE_SCORE_TIF) as src:
    slope_score = src.read(1)
    slope_score[slope_score == -9999] = np.nan
    criteria_data['slope_score'] = slope_score

# Load Grid Distance
print("Loading Grid Distance...")
with rasterio.open(GRID_DIST_TIF) as src:
    grid_dist = src.read(1)
    grid_dist[grid_dist == -9999] = np.nan
    criteria_data['grid_distance_km'] = grid_dist / 1000

# Load Roads Distance
print("Loading Roads Distance...")
with rasterio.open(ROADS_DIST_TIF) as src:
    roads_dist = src.read(1)
    roads_dist[roads_dist == -9999] = np.nan
    criteria_data['roads_distance_km'] = roads_dist / 1000

# Load Saturation Score
print("Loading Saturation Score...")
with rasterio.open(SATURATION_SCORE_TIF) as src:
    saturation_score = src.read(1)
    saturation_score[saturation_score == -9999] = np.nan
    criteria_data['saturation_score'] = saturation_score

print("All data layers loaded")

# Step 4: Create candidate sites
print("\n" + "="*80)
print("STEP 4: CREATE CANDIDATE SITES")
print("="*80)

candidate_sites = []

for r in range(rows):
    for c in range(cols):
        wlc = wlc_data[r, c]
        
        if np.isnan(wlc):
            continue
        
        x = x_min + (c + 0.5) * x_spacing
        y = y_max - (r + 0.5) * y_spacing
        
        wind_speed = criteria_data['wind_speed_ms'][r, c]
        wind_score = criteria_data['wind_score'][r, c]
        slope_deg = criteria_data['slope_degrees'][r, c]
        slope_score = criteria_data['slope_score'][r, c]
        grid_dist = criteria_data['grid_distance_km'][r, c]
        roads_dist = criteria_data['roads_distance_km'][r, c]
        saturation = criteria_data['saturation_score'][r, c]
        
        candidate_sites.append({
            'x': x,
            'y': y,
            'wlc_score': wlc,
            'wind_speed_ms': wind_speed,
            'wind_score': wind_score,
            'slope_degrees': slope_deg,
            'slope_score': slope_score,
            'grid_distance_km': grid_dist,
            'roads_distance_km': roads_dist,
            'saturation_score': saturation
        })

sites_df = pd.DataFrame(candidate_sites)
print(f"Total candidate sites: {len(sites_df):,}")

# Step 5: Apply filters
print("\n" + "="*80)
print("STEP 5: APPLY FILTERS")
print("="*80)

# 5.1: Northern Ireland exclusion
print("\n5.1: Exclude Northern Ireland")
ni_mask = (
    (sites_df['x'] >= NI_BOUNDARY['x_min']) &
    (sites_df['x'] <= NI_BOUNDARY['x_max']) &
    (sites_df['y'] >= NI_BOUNDARY['y_min']) &
    (sites_df['y'] <= NI_BOUNDARY['y_max'])
)
ni_count = ni_mask.sum()
sites_df = sites_df[~ni_mask].copy()
print(f"  Excluded NI sites: {ni_count:,}")
print(f"  Remaining GB sites: {len(sites_df):,}")

# 5.2: Roads distance filter
print(f"\n5.2: Filter by roads distance ≤ {MAX_ROADS_DISTANCE_KM} km")
roads_mask = sites_df['roads_distance_km'] <= MAX_ROADS_DISTANCE_KM
remote_count = (~roads_mask).sum()
sites_df = sites_df[roads_mask].copy()
print(f"  Excluded remote/offshore sites: {remote_count:,}")
print(f"  Remaining accessible sites: {len(sites_df):,}")

# 5.3: Wind speed threshold (NEW)
print(f"\n5.3: Filter by wind speed ≥ {MIN_WIND_SPEED_MS} m/s (economic viability)")
before_wind = len(sites_df)
wind_mask = sites_df['wind_speed_ms'] >= MIN_WIND_SPEED_MS
sites_df = sites_df[wind_mask].copy()
after_wind = len(sites_df)
print(f"  Excluded low wind sites: {before_wind - after_wind:,}")
print(f"  Remaining viable sites: {after_wind:,}")

# Step 6: Select top 20 with separation
print("\n" + "="*80)
print("STEP 6: SELECT TOP 20 SITES (WITH SEPARATION)")
print("="*80)

sites_df_sorted = sites_df.sort_values('wlc_score', ascending=False).reset_index(drop=True)

print(f"Applying {MIN_SEPARATION_KM} km minimum separation...")

selected_sites = []
separation_m = MIN_SEPARATION_KM * 1000

for idx, candidate in sites_df_sorted.iterrows():
    too_close = False
    
    for selected in selected_sites:
        distance = np.sqrt(
            (candidate['x'] - selected['x'])**2 + 
            (candidate['y'] - selected['y'])**2
        )
        
        if distance < separation_m:
            too_close = True
            break
    
    if not too_close:
        selected_sites.append(candidate)
        
        if len(selected_sites) >= TOP_N:
            break

print(f"Selected {len(selected_sites)} sites with {MIN_SEPARATION_KM} km separation")

# Step 7: Create output DataFrame
print("\n" + "="*80)
print("STEP 7: PREPARE OUTPUT DATA")
print("="*80)

top20_df = pd.DataFrame(selected_sites)
top20_df['rank'] = range(1, len(top20_df) + 1)

# Reorder columns
top20_df = top20_df[[
    'rank', 'x', 'y', 'wlc_score', 
    'wind_speed_ms', 'wind_score',
    'slope_degrees', 'slope_score',
    'grid_distance_km', 'roads_distance_km',
    'saturation_score'
]]

# Round for readability
top20_df = top20_df.round({
    'wlc_score': 2,
    'wind_speed_ms': 2,
    'wind_score': 2,
    'slope_degrees': 2,
    'slope_score': 2,
    'grid_distance_km': 2,
    'roads_distance_km': 2,
    'saturation_score': 2
})

print(f"\nTop 20 Sites Summary:")
print(f"  WLC Score: {top20_df['wlc_score'].mean():.2f} ± {top20_df['wlc_score'].std():.2f}")
print(f"  Wind Speed: {top20_df['wind_speed_ms'].mean():.2f} ± {top20_df['wind_speed_ms'].std():.2f} m/s")
print(f"    Range: {top20_df['wind_speed_ms'].min():.2f} - {top20_df['wind_speed_ms'].max():.2f} m/s")
print(f"  Slope: {top20_df['slope_degrees'].mean():.2f} ± {top20_df['slope_degrees'].std():.2f} degrees")
print(f"  Saturation: {top20_df['saturation_score'].mean():.2f} ± {top20_df['saturation_score'].std():.2f}")
print(f"  Grid Distance: {top20_df['grid_distance_km'].mean():.2f} ± {top20_df['grid_distance_km'].std():.2f} km")
print(f"  Roads Distance: {top20_df['roads_distance_km'].mean():.2f} ± {top20_df['roads_distance_km'].std():.2f} km")

# Regional distribution
scotland_sites = top20_df[top20_df['y'] > 600000]
england_wales_sites = top20_df[top20_df['y'] <= 600000]
print(f"\nRegional Distribution:")
print(f"  Scotland: {len(scotland_sites)} sites ({len(scotland_sites)/len(top20_df)*100:.0f}%)")
print(f"  England/Wales: {len(england_wales_sites)} sites ({len(england_wales_sites)/len(top20_df)*100:.0f}%)")

# Step 8: Save outputs
print("\n" + "="*80)
print("STEP 8: SAVE OUTPUTS")
print("="*80)

top20_df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved: {OUTPUT_CSV}")

top20_df.to_csv(OUTPUT_DETAILED_CSV, index=False)
print(f"Saved: {OUTPUT_DETAILED_CSV}")

# Step 9: Visualization
print("\n" + "="*80)
print("STEP 9: GENERATE VISUALIZATION MAP")
print("="*80)

uk_boundary = gpd.read_file(UK_BOUNDARY_SHP)
if uk_boundary.crs.to_epsg() != 27700:
    uk_boundary = uk_boundary.to_crs('EPSG:27700')

fig, ax = plt.subplots(figsize=(12, 16), dpi=300)
ax.set_facecolor('#D6EAF8')

uk_boundary.plot(ax=ax, color='#FFF8DC', edgecolor='#424242', linewidth=1, alpha=1.0, zorder=1)

wlc_plot = np.where(np.isnan(wlc_data), -9999, wlc_data)
im = ax.imshow(wlc_plot, extent=extent, origin='upper', cmap='RdYlGn', 
               vmin=0, vmax=10, alpha=0.4, zorder=2)

for idx, site in top20_df.iterrows():
    color = plt.cm.RdYlGn(site['wlc_score'] / 10)
    
    ax.plot(site['x'], site['y'], 'o', color=color, markersize=12, 
            markeredgecolor='black', markeredgewidth=2, zorder=10)
    
    circle = Circle((site['x'], site['y']), MIN_SEPARATION_KM * 1000, 
                   fill=False, edgecolor='red', linewidth=0.5, 
                   linestyle='--', alpha=0.3, zorder=5)
    ax.add_patch(circle)
    
    ax.text(site['x'], site['y'], str(site['rank']), 
           fontsize=8, fontweight='bold', ha='center', va='center',
           color='white', zorder=11,
           bbox=dict(boxstyle='circle', facecolor='black', alpha=0.7, pad=0.3))

ax.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax.set_title(f'Top {TOP_N} Optimal Wind Farm Sites (5-Criteria WLC)\n' +
             f'Wind Speed ≥ {MIN_WIND_SPEED_MS} m/s | Regional Saturation Constraint\n' +
             f'Minimum Separation: {MIN_SEPARATION_KM} km | Geographic Scope: Great Britain',
             fontsize=14, fontweight='bold', pad=20)

ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])
ax.set_aspect('equal', adjustable='box')

from matplotlib.ticker import FuncFormatter
def format_km(value, tick_number):
    return f'{int(value/1000)}'

ax.xaxis.set_major_formatter(FuncFormatter(format_km))
ax.yaxis.set_major_formatter(FuncFormatter(format_km))

cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('WLC Suitability Score', fontsize=11, fontweight='bold')

from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', 
           markersize=10, markeredgecolor='black', markeredgewidth=2,
           label=f'Top {TOP_N} Sites'),
    Line2D([0], [0], color='red', linestyle='--', linewidth=1,
           label=f'{MIN_SEPARATION_KM} km separation'),
]

ax.legend(handles=legend_elements, loc='upper left', fontsize=10, framealpha=0.95)

stats_text = f"""Top {TOP_N} Sites:

WLC: {top20_df['wlc_score'].mean():.2f}
  ({top20_df['wlc_score'].min():.2f}-{top20_df['wlc_score'].max():.2f})

Wind: {top20_df['wind_speed_ms'].mean():.2f} m/s
  ({top20_df['wind_speed_ms'].min():.2f}-{top20_df['wind_speed_ms'].max():.2f})

Slope: {top20_df['slope_degrees'].mean():.2f}°
Saturation: {top20_df['saturation_score'].mean():.2f}
Grid: {top20_df['grid_distance_km'].mean():.1f} km
Roads: {top20_df['roads_distance_km'].mean():.1f} km

Regional:
Scotland: {len(scotland_sites)}
Eng/Wales: {len(england_wales_sites)}

Criteria Weights:
Wind: 37%  Grid: 37%
Roads: 13%  Slope: 7%
Saturation: 6%"""

props = dict(boxstyle='round', facecolor='white', alpha=0.95, edgecolor='black', linewidth=1.5)
ax.text(0.02, 0.02, stats_text, transform=ax.transAxes,
        fontsize=8, verticalalignment='bottom', horizontalalignment='left',
        bbox=props, fontfamily='monospace')

scale_x = extent[0] + (extent[1] - extent[0]) * 0.70
scale_y = extent[2] + (extent[3] - extent[2]) * 0.05
scale_length = 100000
ax.plot([scale_x, scale_x + scale_length], [scale_y, scale_y], 'k-', linewidth=3, zorder=15)
ax.text(scale_x + scale_length/2, scale_y + 10000, '100 km',
        ha='center', fontsize=10, fontweight='bold', zorder=15)

arrow_x = extent[1] - (extent[1] - extent[0]) * 0.05
arrow_y = extent[3] - (extent[3] - extent[2]) * 0.05
ax.annotate('N', xy=(arrow_x, arrow_y), xytext=(arrow_x, arrow_y - 40000),
            arrowprops=dict(arrowstyle='->', lw=3, color='black'),
            fontsize=14, fontweight='bold', ha='center', zorder=15)

plt.tight_layout()
plt.savefig(OUTPUT_MAP, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Saved: {OUTPUT_MAP}")
plt.close()

# Step 10: Print results table
print("\n" + "="*80)
print("TOP 20 OPTIMAL ONSHORE WIND FARM SITES (GREAT BRITAIN)")
print("="*80)

print(f"\nRank  Easting   Northing    WLC   Wind  Slope   Grid  Roads  Satur")
print(f"      (m)        (m)       Score (m/s)  (deg)   (km)   (km)  (scr)")
print("-" * 80)

for _, row in top20_df.iterrows():
    print(f"{row['rank']:2.0f}    {row['x']:7.0f}   {row['y']:8.0f}   "
          f"{row['wlc_score']:5.2f} {row['wind_speed_ms']:5.2f} {row['slope_degrees']:5.2f}  "
          f"{row['grid_distance_km']:5.2f} {row['roads_distance_km']:6.2f} {row['saturation_score']:5.2f}")

print("\n" + "="*80)
print("SITE SELECTION COMPLETE")
print("="*80)

print(f"""
Key Filters Applied:
  ✓ Wind speed ≥ {MIN_WIND_SPEED_MS} m/s (economic viability)
  ✓ Roads distance ≤ {MAX_ROADS_DISTANCE_KM} km (mainland accessible)
  ✓ Northern Ireland excluded (GB scope)
  ✓ {MIN_SEPARATION_KM} km minimum separation
  ✓ Hard constraints (protected areas, settlements, water)

Statistics:
  Mean Wind Speed: {top20_df['wind_speed_ms'].mean():.2f} m/s (range: {top20_df['wind_speed_ms'].min():.2f}-{top20_df['wind_speed_ms'].max():.2f})
  Mean Slope: {top20_df['slope_degrees'].mean():.2f}° (range: {top20_df['slope_degrees'].min():.2f}-{top20_df['slope_degrees'].max():.2f})
  Mean Saturation: {top20_df['saturation_score'].mean():.2f}

All sites meet economic viability threshold (≥6 m/s)!
""")

TOP 20 OPTIMAL ONSHORE WIND FARM SITES
Based on 5-Criteria WLC (including Regional Saturation)
With Wind Speed ≥ 6 m/s Threshold

Selection Parameters:
  Top N sites: 20
  Minimum separation: 35 km
  Maximum roads distance: 30 km
  Minimum wind speed: 6.0 m/s (economic viability)
  Northern Ireland: EXCLUDED
  Geographic scope: Great Britain (England, Scotland, Wales)

STEP 1: LOAD REFERENCE GRID
Grid: 235 x 131
Pixel size: 5000m x 5000m

STEP 2: LOAD CONSTRAINED WLC (5 CRITERIA)
Valid cells: 2,599
WLC range: 0.66 - 8.77

STEP 3: LOAD DATA LAYERS
Loading Wind Speed (m/s)...
Loading Wind Score...
Loading Slope (degrees)...
Loading Slope Score...
Loading Grid Distance...
Loading Roads Distance...
Loading Saturation Score...
All data layers loaded

STEP 4: CREATE CANDIDATE SITES
Total candidate sites: 2,599

STEP 5: APPLY FILTERS

5.1: Exclude Northern Ireland
  Excluded NI sites: 454
  Remaining GB sites: 2,145

5.2: Filter by roads distance ≤ 30 km
  Excluded remote/offshore sites: 214
